In [1]:
import os,sys
project_path = os.path.abspath(os.path.join('..'))
sys.path.append(project_path)
os.environ.get("CURRENTS_API_KEY")
print("XXXXXXXXXX=>", os.environ.get("CURRENTS_API_KEY"))
currentapi_api_key = os.environ.get("CURRENTS_API_KEY")
Connection_String = os.environ.get("Connection_String")

XXXXXXXXXX=> kgqtONncY5F_VA-pKQoFT0WZCyXSkuDrs2T0vDdEAc5Ty8Pg


In [2]:
from src.etl.extract.data_structure_extract_strategy import DataExtractor

config = {
    "type": "api_url",
    "path": "https://api.currentsapi.services/v1/latest-news",
    "params": {"language": "en", "apiKey": currentapi_api_key},
}
print("XXXXXXXXXX=>", currentapi_api_key)
data = DataExtractor.get_extractor(config).extractor()

XXXXXXXXXX=> kgqtONncY5F_VA-pKQoFT0WZCyXSkuDrs2T0vDdEAc5Ty8Pg


In [3]:
from src.etl.transform.data_format_strategy import (
    FromJsonToDataFrameFormatter,
    FromDataFrameToCsvFormatter,
    DataFormatter,
    )
from src.etl.transform.data_metadata_strategy import MetadataAppender
from datetime import datetime, timedelta, timezone
import pandas as pd


df = DataFormatter(
FromJsonToDataFrameFormatter(array_keys=["news"])
).formatting(data)
metadata = {
"createdate": pd.Timestamp.now(tz=timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
"usercreate": "system",
"updatedate": pd.NaT,
"userupdate": "system",
"activedata": True,
"batch_id": "run_id",
"source": "currentsapi",
"layer": "bronze",
}
meta_appender = MetadataAppender(metadata)
df = meta_appender.meta(df)
csv = DataFormatter(
FromDataFrameToCsvFormatter(index=False, encoding="utf-8-sig", sep=",")
).formatting(df)

In [4]:
df.head()

,id,title,description,url,author,image,language,category,published,createdate,usercreate,updatedate,userupdate,activedata,batch_id,source,layer
0,ad829cae-e9e6-4a45-bcd0-654bacdd39da,Rick Beato: Finally…Breaking Down Kansas LIVE,Finally…Breaking Down Kansas LIVE unpacks Rick...,https://music.forem.com/music_youtube/rick-bea...,Music YouTube,https://media2.dev.to/dynamic/image/width=1000...,en,[programming],2025-10-06 16:08:53 +0000,2025-10-06T16:16:44Z,system,NaT,system,True,run_id,currentsapi,bronze
1,81dea338-ea9a-4cb9-8531-be77734d9849,How I Rendered Mermaid Diagrams in React (and ...,🧠 TL;DR — I couldn't find a working React libr...,https://dev.to/navdeepm20/how-i-rendered-merma...,Navdeep Mishra,None,en,[programming],2025-10-06 16:08:09 +0000,2025-10-06T16:16:44Z,system,NaT,system,True,run_id,currentsapi,bronze
2,3d149351-a30c-4944-9ba2-e13b1ff3eb13,GameSpot: Call of Duty Developers Share Their ...,Watch on YouTube,https://gg.forem.com/gg_news/gamespot-call-of-...,Gaming News,https://media2.dev.to/dynamic/image/width=1000...,en,[programming],2025-10-06 16:03:16 +0000,2025-10-06T16:16:44Z,system,NaT,system,True,run_id,currentsapi,bronze
3,85a05a1c-54ee-45a5-a77c-7a3370e3cd5f,GameSpot: How Can EA Save Its Reputation?,Watch on YouTube,https://gg.forem.com/gg_news/gamespot-how-can-...,Gaming News,https://media2.dev.to/dynamic/image/width=1000...,en,[programming],2025-10-06 16:03:06 +0000,2025-10-06T16:16:44Z,system,NaT,system,True,run_id,currentsapi,bronze
4,64687ffc-5e03-4627-956d-4f68d970e07c,IGN: Your Host - Official Trailer #2 (2025) Ja...,Your Host – Official Trailer #2 (2025) drops y...,https://gg.forem.com/gg_news/ign-your-host-off...,Gaming News,https://media2.dev.to/dynamic/image/width=1000...,en,[programming],2025-10-06 16:02:54 +0000,2025-10-06T16:16:44Z,system,NaT,system,True,run_id,currentsapi,bronze


In [5]:
from src.etl.load.data_structure_loader_strategy import PostgresCsvCopyLoader
loader = PostgresCsvCopyLoader(
            Connection_String,
            "bronze",
            "source_news_articles_currentsapi",
        )
loader.create(csv)

{'status': 'success', 'table': 'bronze.source_news_articles_currentsapi'}